In [1]:
from qwanta import Tuner, QuantumNetwork, Configuration, Xperiment
from qwanta import QuantumNetwork, Configuration, Xperiment, Tuner
import pandas as pd
import ast
import numpy as np
import networkx as nx
from qwanta.Qubit import PhysicalQubit
import seaborn as sns
import matplotlib.pyplot as plt

## Define configuration and read the network timeline

As of current implementation, the timeline should be format as in csv or excel file to let Xperiment to be able to read it. However, Xperiment is just a class wrapping that does the validation of the timeline and the network, so you can use any other format as long as you can read it and pass it to the Xperiment class.

In [12]:
# Hyperparameters

loss = 0.1 # dB/km
depo_prob = 0
gate_error = 0.1 
measurement_error = 0.1
memory_time = np.inf # second

num_hops = 2
num_nodes = num_hops + 1

node_info = {f'Node {i}': {'coordinate': (int(i*100), 0, 0)} for i in range(num_nodes)}
edge_info = {
    (f'Node {i}', f'Node {i+1}'): {
    'connection-type': 'Space',
    'depolarlizing error': [1 - depo_prob, depo_prob/3, depo_prob/3, depo_prob/3],
    'loss': loss,
    'light speed': 300000,
    'Pulse rate': 0.0001,
    f'Node {i}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    f'Node {i+1}':{
        'gate error': gate_error,
        'measurement error': measurement_error,
        'memory function': memory_time
    },
    }
for i in range(num_hops)}

exps = Xperiment(
    timelines_path = 'network/exper_id3_selectedStats_2hops.xlsx',
    nodes_info_exp = node_info,
    edges_info_exp = edge_info,
    gate_error = gate_error,
    measurement_error = measurement_error,
    memory_time = memory_time
)

## To validate the experiment configuration, and network topology

You can validate the experiment configuration and network topology by running the following command. If vis is set to True, the network topology will be visualized using pyvis library, you could see the network topology in the browser using the generated html file. If get_table is set to True, the network topology will be printed in table format show the summary of validation. If show_message is set to True, the validation message will be printed.

In [3]:
exps.validate(get_table=True, vis=True)

,Experiment,Resource-reachable,loss,depolarizing error,gate error,memory error,measurement error,Network
0,0G,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
1,1G-Ss-Dp,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
2,2G-NonLocalCNOT,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
3,1-2G-DirectedEncoded,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
4,HG-END2ENDPurifiedEncoded,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
5,1-2G-DirectedEncoded-Perfect,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED
6,2G-NonLocalCNOT-Perfect,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED,PASSED


## To run the experiment

You could run the experiment by running the following command. Set save_result to True to save the experiment result to the experiment result folder.

In [13]:
results = exps.execute()

KeyboardInterrupt: 

## Let us see the result of 0G experiment

The result dictionary contains the following keys:

In [5]:
for key in results['0G'].keys():
    print(key)

numPhysicalBuffer
numInternalEncodingBuffer
numInternalDetectingBuffer
numInternalInterfaceBuffer
memory_function
gate_error
measurement_error
timeline
experiment
light_speed_in_fiber
message
g
result_path
label_recorded
collectFidelityHistory
simulation_time
coor_system
nodes_info
NetworkTopology
throughtputEdges
fidelity
Resources Produced
Base Resources Attempt
Resource Used in Fidelity Estimation
Time used
Fidelity Estimation Time
Fidelity History
Qubits waiting time
Commutation inforamtion
Simulation log


In [6]:
def visualize(result):
    to_print = ['fidelity', 'simulation_time', 'Resources Produced', 'Base Resources Attempt', 'Resource Used in Fidelity Estimation', 'Time used', 'Fidelity Estimation Time']
    for key in to_print:
        print(f'{key}: {result[key]}')

In [7]:
visualize(results['0G'])

fidelity: 1.0
simulation_time: None
Resources Produced: {'Node 0-Node 1': {'Physical': 9004}, 'Node 1-Node 2': {'Physical': 9004}, 'Node 0-Node 2': {'k': 9001}}
Base Resources Attempt: 18020
Resource Used in Fidelity Estimation: 9000
Time used: 0.900999999999917
Fidelity Estimation Time: 0.900899999999917


In [10]:
results['0G']['fidelity']

1.0

In [11]:
protocol = [
    '0G', 
    '1G-Ss-Dp', 
    '1-2G-DirectedEncoded', 
    '1-2G-DirectedEncoded-Perfect', 
    '2G-NonLocalCNOT',
    '2G-NonLocalCNOT-Perfect',
]

for p in protocol:
    print(f'Protocol: {p}')
    visualize(results[p])
    print('- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -')

Protocol: 0G
fidelity: 1.0
simulation_time: None
Resources Produced: {'Node 0-Node 1': {'Physical': 9004}, 'Node 1-Node 2': {'Physical': 9004}, 'Node 0-Node 2': {'k': 9001}}
Base Resources Attempt: 18020
Resource Used in Fidelity Estimation: 9000
Time used: 0.900999999999917
Fidelity Estimation Time: 0.900899999999917
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Protocol: 1G-Ss-Dp
fidelity: 1.0
simulation_time: None
Resources Produced: {'Node 0-Node 1': {'Physical': 27009, 'Purified': 9002}, 'Node 1-Node 2': {'Physical': 27009, 'Purified': 9002}, 'Node 0-Node 2': {'k': 9001}}
Base Resources Attempt: 54030
Resource Used in Fidelity Estimation: 9000
Time used: 2.70153333333461
Fidelity Estimation Time: 2.7012333333346095
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Protocol: 1-2G-DirectedEncoded
fidelity: 1.0
simulation_time: None
Resources Produced: {'Node 0-Node 1': {'Physical': 27009, 'Purified': 9002, 'Logi